# Mean Absolute errors


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# do we have the correct env loaded?
assert 'donkey2' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [3]:
import numpy as np
fast_model_fast_data_3d = np.load('nighttime-fast-auto-3d-model-nighttime-synth-fast-synth-auto-data-prediction-and-ground-truth.npy')
fast_model_slow_data_3d = np.load('nighttime-fast-auto-3d-model-nighttime-synth-slow-auto-data-prediction-and-ground-truth.npy')
slow_model_fast_data_3d = np.load('nighttime-slow-auto-3d-model-nighttime-fast-synth-auto-data-prediction-and-ground-truth.npy')
slow_model_slow_data_3d = np.load('nighttime-slow-auto-3d-model-nighttime-slow-auto-data-prediction-and-ground-truth.npy')



In [4]:
preds_and_ground_truths = dict(
    fast_fast_3d=fast_model_fast_data_3d,
    slow_fast_3d=slow_model_fast_data_3d,
    fast_slow_3d=fast_model_slow_data_3d,
    slow_slow_3d=slow_model_slow_data_3d,
)

In [5]:
import pandas as pd

records = []

for name, data in preds_and_ground_truths.items():
    model_speed = name.split('_')[0]
    model_type = name.split('_')[2]
    data_speed = name.split('_')[1]
    for (gt, pred) in zip(data[:, data.shape[1] - 1], data[:, data.shape[1] - 2]):
        dd = dict()
        dd['model speed'] = model_speed
        dd['model type'] = model_type
        dd['data speed'] = data_speed
        dd['ground_truth_angle'] = gt
        dd['predicted_angle'] = pred
        records.append(dd)
df = pd.DataFrame.from_records(records)

df.to_csv('gt_and_preds.csv.gz')
df

,model speed,model type,data speed,ground_truth_angle,predicted_angle
0,fast,3d,fast,-0.170407,-0.179481
1,fast,3d,fast,-0.135117,-0.126676
2,fast,3d,fast,-0.084213,-0.096450
3,fast,3d,fast,-0.036450,-0.033739
4,fast,3d,fast,0.005086,0.045538
...,...,...,...,...,...
40595,slow,3d,slow,-0.448041,-0.488623
40596,slow,3d,slow,-0.389852,-0.442383
40597,slow,3d,slow,-0.368167,-0.426429
40598,slow,3d,slow,-0.345837,-0.398514


In [6]:
df['mean absolute ground truth angle'] = abs(df.ground_truth_angle)
df['mean absolute predicted angle'] = abs(df.predicted_angle)
df['mean absolute error'] = abs(df.predicted_angle - df.ground_truth_angle)
df = df.drop(columns=['ground_truth_angle', 'predicted_angle'])

In [7]:
df.groupby(['model speed', 'model type', 'data speed']).mean()

mean absolute ground truth angle  \
model speed model type data speed                                     
fast        3d         fast                                0.348779   
                       slow                                0.348622   
slow        3d         fast                                0.348779   
                       slow                                0.348622   

                                   mean absolute predicted angle  \
model speed model type data speed                                  
fast        3d         fast                             0.340130   
                       slow                             0.336579   
slow        3d         fast                             0.269072   
                       slow                             0.337147   

                                   mean absolute error  
model speed model type data speed                       
fast        3d         fast                   0.056652  
                       slow                   0.058538  
slow        3d         fast                   0.096206  
                       slow                   0.085675